# Librerías

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

# Datos

In [2]:
df = pd.read_csv("../Datos/RMCAB/Clean/Sec_Cruzada_RMCAB.csv")
df["Fecha_Hora"] = pd.to_datetime(df["Fecha_Hora"])

In [3]:
df.head(3)

,Fecha_Hora,Estación,BBP,BC ug/m3,CO,Dir Viento,Dir Viento 10M,HR,HR_2m,Humedad Inter,...,SO2,SO2 Envea,Temp_4m,Tempe Inter,Temperatura,Temperatura 20M,Temperatura 8M,UV-BC,Vel Viento,Vel Viento 10M
0,2000-01-01,Carvajal - Sevillana,NaN,NaN,0.7,306.0,NaN,NaN,NaN,NaN,...,2.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,NaN
1,2000-01-01,Guaymaral,NaN,NaN,NaN,NaN,194.0,94.01,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,2000-01-01,MinAmbiente,NaN,NaN,NaN,293.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.4,NaN


# Identificación de estaciones

> Filtrar las estaciones con datos desde el 2000

In [4]:
estaciones = df["Estación"].unique().tolist()
info_estaciones = pd.DataFrame(columns=["Estación", "Min_Fecha_Hora"])

# Iterate over each station and find the minimum date and time
for estacion in estaciones:
    min_date = df[df["Estación"] == estacion]["Fecha_Hora"].min()
    new_row = pd.DataFrame({"Estación": [estacion], "Min_Fecha_Hora": [min_date]})
    info_estaciones = pd.concat([info_estaciones, new_row], ignore_index=True)
info_estaciones

,Estación,Min_Fecha_Hora
0,Carvajal - Sevillana,2000-01-01 00:00:00
1,Guaymaral,2000-01-01 00:00:00
2,MinAmbiente,2000-01-01 00:00:00
3,Puente Aranda,2000-01-01 00:00:00
4,Suba,2000-01-01 00:00:00
5,Usaquen,2000-01-01 00:00:00
6,Las Ferias,2000-02-22 00:00:00
7,Centro de Alto Rendimiento,2003-01-11 00:00:00
8,Kennedy,2005-10-03 11:00:00
9,Tunal,2006-02-16 01:00:00


In [5]:
estaciones_filtradas = info_estaciones.head(7)["Estación"].tolist()
estaciones_filtradas

['Carvajal - Sevillana',
 'Guaymaral',
 'MinAmbiente',
 'Puente Aranda',
 'Suba',
 'Usaquen',
 'Las Ferias']

# Info variables por estación

In [6]:
# Filtrar datos por las estaciones con datos desde el 2000
df = df[df["Estación"].isin(estaciones_filtradas)]
variables = df.columns[2:].tolist()

In [7]:
try:
    df.set_index("Fecha_Hora", inplace=True)
except:
    pass

fechas_variables = []

for estacion in estaciones_filtradas:
    for var in variables:
        var_df = pd.DataFrame((df.loc[df["Estación"]==estacion])[var])
        var_df = var_df.dropna()
        min_date = var_df.index.min()
        fechas_variables.append({
            "Estación": estacion,
            "Variable": var,
            "Fecha Inicio": min_date
        })

# Convertir la lista de diccionarios en un DataFrame
fechas_variables_df = pd.DataFrame(fechas_variables)
fechas_variables_df = fechas_variables_df.pivot_table(index="Estación", columns="Variable", values="Fecha Inicio")
fechas_variables_df

Variable,BBP,BC ug/m3,CO,Dir Viento,Dir Viento 10M,HR,Humedad Inter,NO,NO2,NOX,...,Precipitacion,Presion Baro,Rad Solar,SO2,Tempe Inter,Temperatura,Temperatura 20M,Temperatura 8M,Vel Viento,Vel Viento 10M
Estación,,,,,,,,,,,,,,,,,,,,,
Carvajal - Sevillana,NaT,NaT,2000-01-01,2000-01-01,NaT,NaT,NaT,2000-01-01,2000-01-01,2000-01-01,...,2000-01-01,NaT,NaT,2000-01-01,NaT,2005-06-06,NaT,NaT,2000-01-01,NaT
Guaymaral,NaT,NaT,2021-06-01,NaT,2000-01-01,2000-01-01,NaT,2008-05-26,2008-05-26,2001-05-01,...,2000-01-01,2000-01-01,2000-01-01,2024-03-04,NaT,2008-02-29,2008-02-29,2008-02-29,NaT,2000-01-01
Las Ferias,NaT,NaT,2005-07-16,2000-06-19,NaT,2007-07-18,2019-01-14,2000-08-17,2000-08-17,2000-08-17,...,2000-02-22,2005-04-03,NaT,NaT,2015-03-16,2000-02-22,NaT,NaT,2000-02-22,NaT
MinAmbiente,NaT,NaT,2020-10-13,2000-01-01,NaT,NaT,NaT,2021-06-01,2021-06-01,2021-06-01,...,2000-01-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2000-01-01,NaT
Puente Aranda,2023-10-01,2023-10-01,2000-01-01,2000-01-01,NaT,2021-12-01,NaT,2000-01-01,2000-01-01,2000-01-01,...,2000-01-01,2021-11-12,2021-11-12,2000-01-01,NaT,NaT,NaT,NaT,2000-01-01,NaT
Suba,NaT,NaT,2020-10-13,2000-01-01,NaT,NaT,NaT,2000-01-01,2000-01-01,2000-01-01,...,2000-01-01,NaT,NaT,2008-05-29,NaT,2000-01-01,NaT,NaT,2000-01-01,NaT
Usaquen,NaT,NaT,2010-08-27,2000-01-01,NaT,NaT,NaT,2000-01-01,2000-01-01,2000-01-01,...,2000-01-01,NaT,NaT,2000-01-01,NaT,2007-08-01,NaT,NaT,2000-01-01,NaT


In [8]:
#fechas_variables_df.to_excel("./Fechas_Inicio_Variables.xlsx")
#correlations_by_estacion = df.groupby("Estación").apply(lambda x: x.drop(columns=["Estación"]).corr())
#correlations_by_estacion

# Análisis exploratorio

## Disponibilidad estatica

> Ver la disponiblidad en porcentaje desde el 2000, así como desde la fecha inicial

In [9]:

df1 = pd.DataFrame(columns=["Estación", "Variable", "Disp_2000", "Disp_inicial", "Fecha_inicial"])

for estacion in estaciones_filtradas:

    e = df[df["Estación"] == estacion]

    disp_2000 = []
    disp_principio = []
    disp_var = []
    fechas_iniciales = []
    for var in variables:

        df_disp = e[var]
        disponibilidad_2000 = df_disp.isna().sum() / len(df_disp)
        fecha_inicial = df_disp.dropna().index.min()
        disp_prin = df_disp.loc[df_disp.index >= fecha_inicial].isna().sum() / len(df_disp.loc[df_disp.index >= fecha_inicial])

        disp_2000.append(1-disponibilidad_2000)
        disp_var.append(var)
        disp_principio.append(1-disp_prin)

        fechas_iniciales.append(fecha_inicial)

    disp_df = pd.DataFrame({"Variable": disp_var, "Disp_2000": disp_2000,"Disp_inicial":disp_principio, "Fecha_inicial":fechas_iniciales}).round(2)
    disp_df["Estación"] = estacion
    disp_df = disp_df.sort_values(by="Disp_inicial", ascending=False)
    disp_df = disp_df.dropna()
    df1 = pd.concat([df1, disp_df], ignore_index=True)
df1

,Estación,Variable,Disp_2000,Disp_inicial,Fecha_inicial
0,Carvajal - Sevillana,Temperatura,0.77,0.98,2005-06-06
1,Carvajal - Sevillana,Precipitacion,0.97,0.97,2000-01-01
2,Carvajal - Sevillana,Dir Viento,0.90,0.90,2000-01-01
3,Carvajal - Sevillana,Vel Viento,0.90,0.90,2000-01-01
4,Carvajal - Sevillana,PM10,0.88,0.88,2000-01-01
...,...,...,...,...,...
90,Las Ferias,PM2.5,0.55,0.81,2008-09-12
91,Las Ferias,OZONO,0.64,0.79,2005-09-26
92,Las Ferias,NO,0.72,0.74,2000-08-17
93,Las Ferias,NOX,0.72,0.74,2000-08-17


## Disponibilidad a lo largo del tiempo

In [10]:
# Crear un dataframe vacío para almacenar resultados
dfs2 = pd.DataFrame(columns=["Estación", "Variable", "Año", "Disponibilidad"])

# Iterar por cada estación
for estacion in estaciones_filtradas:
    
    # Filtrar datos por estación
    e = df[df["Estación"] == estacion]
    
    # Iterar por cada variable
    for var in variables:
        
        # Extraer los datos de la variable en la estación correspondiente
        df_disp = e[var]
        
        # Iterar por cada año presente en los datos
        for year in df_disp.index.year.unique():
            
            # Filtrar los datos por año
            df_year = df_disp[df_disp.index.year == year]
            
            # Calcular el porcentaje de datos no nulos
            disponibilidad_anual = df_year.notna().sum() / len(df_year)
            
            # Crear un nuevo registro con los resultados
            disp_df = pd.DataFrame({
                "Estación": [estacion],
                "Variable": [var],
                "Año": [year],
                "Disponibilidad": [disponibilidad_anual]
            })
            
            # Concatenar el resultado al dataframe general
            dfs2 = pd.concat([dfs2, disp_df], ignore_index=True)

# Redondear los resultados a 2 decimales
dfs2 = dfs2.round(2)
dfs2["Año"] = dfs2["Año"].astype(int)
dfs2 = dfs2.loc[dfs2["Disponibilidad"] > 0]
dfs2

,Estación,Variable,Año,Disponibilidad
50,Carvajal - Sevillana,CO,2000,0.92
51,Carvajal - Sevillana,CO,2001,0.56
52,Carvajal - Sevillana,CO,2002,0.27
53,Carvajal - Sevillana,CO,2003,0.90
54,Carvajal - Sevillana,CO,2004,0.12
...,...,...,...,...
4758,Las Ferias,Vel Viento,2020,0.93
4759,Las Ferias,Vel Viento,2021,1.00
4760,Las Ferias,Vel Viento,2022,1.00
4761,Las Ferias,Vel Viento,2023,0.97


## Visualizaciones 

### Serie de tiempo

In [11]:
#for estacion in estaciones_filtradas:
#    
#    vis_df = dfs[dfs["Estación"] == estacion]
#    for var in vis_df["Variable"].tolist():
#
#        plt.figure(figsize=(10, 6))
#        plt.plot(df.loc[df["Estación"] == estacion, var])
#        plt.title(f"{estacion} - {var}")
#        plt.show()

### Disponibilidad

In [12]:

#dfs["Año"] = pd.to_datetime(dfs["Año"])
#dfs = dfs.loc[dfs["Año"] != "2024"]
#for estacion in estaciones_filtradas:
#    vis_df = dfs.loc[dfs["Estación"] == estacion]
#    for var in vis_df["Variable"].unique():
#        var_df = vis_df[vis_df["Variable"] == var]
#        plt.figure(figsize=(10, 6))
#        plt.bar(var_df["Año"], var_df["Disponibilidad"])
#        plt.title(f"Disponibilidad por año en {estacion} - {var}")
#        plt.xlabel("Año")
#        plt.ylabel("Disponibilidad")
#        plt.grid(True)
#        plt.show()

In [13]:
# Filtrar para excluir el año 2024
dfs2 = dfs2.loc[dfs2["Año"] != "2024"]
#dfs2["Año"] = dfs2["Año"].astype(int)
#dfs2["Año"] = pd.to_datetime(dfs2["Año"], format="%Y")
dfs2["Año"] = dfs2["Año"].astype(object)
# Definir el número de columnas
cols = 5

# Iterar por cada estación
#for estacion in estaciones_filtradas:
#    # Filtrar los datos por estación
#    vis_df = dfs2.loc[dfs2["Estación"] == estacion]
#    
#    # Obtener las variables únicas para esa estación
#    variables = vis_df["Variable"].unique()
#    
#    # Calcular el número de filas necesarias
#    rows = int(np.ceil(len(variables) / cols))
#    
#    # Crear una figura con subplots, distribuidos en 'rows' filas y 'cols' columnas
#    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(30, 18))
#    
#    # Aplanar el array de axes para facilitar la iteración
#    axes = axes.flatten()
#    
#    # Iterar por cada variable y graficar
#    for i, var in enumerate(variables):
#        # Filtrar los datos por variable
#        var_df = vis_df[vis_df["Variable"] == var]
#        
#        # Graficar barras para la variable en el subplot correspondiente
#        axes[i].bar(var_df["Año"], var_df["Disponibilidad"])
#        
#        # Añadir título y etiquetas
#        axes[i].set_title(f"{var}")
#        axes[i].set_xlabel("Año")
#        axes[i].set_ylabel("Disponibilidad")
#        axes[i].grid(True)
#    
#    # Si hay subplots vacíos (cuando el número de variables no es múltiplo de 5)
#    for j in range(i + 1, len(axes)):
#        fig.delaxes(axes[j])  # Eliminar los subplots sobrantes
#    
#    # Ajustar el layout para evitar solapamientos
#    plt.suptitle(f"Disponibilidad por año en {estacion}", fontsize=16)
#    plt.tight_layout(rect=[0, 0, 1, 0.96])  # Ajustar para no cortar el título
#    
#    # Mostrar la figura con todos los subplots
#    plt.show()

## Imputación para cada variable por estación

In [14]:
df1 = df1.loc[df1["Estación"] == "Guaymaral"].sort_values(by="Disp_inicial", ascending=False)
df1 = df1.loc[df1["Disp_inicial"] > 0.7]

var_to_imput = df1["Variable"].tolist()

## Ejemplo estación

In [ ]:
e = df[df["Estación"] == "Guaymaral"]
e = e.loc[e.index >= "2021-01-01"]
e = e.dropna(axis=1, how='all')
var_to_imput = e.columns[1:].tolist()

In [ ]:
# Iterar sobre cada variable a imputar
for var in var_to_imput:
    disp = 1 - (e[var].isna().sum() / len(e[var]))  # Calcular la disponibilidad de datos
    print(var, disp, len(e[var]))
    # Imputar solo si la disponibilidad de datos es suficiente
    if disp >= 0.7:    
        min_date = e[var].dropna().index.min()
        df_to_replace = e[var].loc[e[var].index >= min_date]
        df_to_replace = pd.DataFrame(df_to_replace)
        #print(df_to_replace)
        n_lags = 50
        correlations = [e[var].autocorr(lag) for lag in range(1, n_lags+1)]
        correlation_df = pd.DataFrame({'Lag': range(1, n_lags+1), 'Correlation': correlations})
        top_lags = correlation_df.sort_values(by='Correlation', ascending=False)
        best_lags = top_lags.head(5)["Lag"].tolist()
        
        imputed_count = 0  # Contador de valores imputados

        # Imputar los valores nulos uno a uno
        while df_to_replace[var].isna().sum() > 0:
            next_nan_value_date = df_to_replace[df_to_replace.isna().any(axis=1)].index[0]
            
            model_df = df_to_replace.loc[df_to_replace.index <= next_nan_value_date]
            model_df = pd.DataFrame(model_df)

            # Crear variables de lag
            for lag in best_lags:
                model_df[f'lag_{lag}'] = model_df[var].shift(lag)

            next_nan_value = model_df.tail(1)
            model_df = model_df.dropna()

            # Preparar los datos de entrenamiento
            X = model_df.drop(columns=[var])
            y = model_df[var]

            # Reentrenar el modelo cada 1000 imputaciones
            if imputed_count % 1000 == 0 or imputed_count == 0:
                model = XGBRegressor()
                model.fit(X, y)
                print(f"Modelo reentrenado en la imputación {imputed_count}")

            # Predecir el siguiente valor nulo y actualizar el DataFrame
            y_pred = model.predict(next_nan_value.drop(columns=var))
            e[var].loc[next_nan_value_date] = y_pred.round(0)
            df_to_replace[var].loc[next_nan_value_date] = y_pred.round(0)

            imputed_count += 1  # Actualizar el contador
            #print(f"Imputación realizada en {next_nan_value_date}: {y_pred.round(0)}")
    if disp < 0.7:
        print(f"La disponibilidad de datos para {var} es insuficiente para imputar")

### Probando método

In [ ]:

# Filtrar la estación y variable de interés (ya hecho previamente)
#e = df.loc[df["Estación"] == "Guaymaral"]
#var = "Presion Baro"
#
## Filtrar la serie temporal (supongamos que no tiene valores nulos actualmente)
#serie_original = e[var].dropna()  # Nos aseguramos de que no haya valores nulos
#serie_original = serie_original.loc[serie_original.index >= "2021-12-31"]  # Filtramos hasta el 31 de diciembre de 2005
#
## Definir la cantidad de datos a utilizar para la validación
#n_validacion = int(len(serie_original) * 0.3)  # 20% de los datos para validación
#serie_validacion = serie_original.tail(n_validacion)  # Seleccionamos el 20% final de la serie
#serie_train = serie_original.head(len(serie_original) - n_validacion)
#
## Establecer un porcentaje de valores faltantes a simular (10% en este caso)
#np.random.seed(0)  # Para reproducibilidad
#n_faltantes = int(len(serie_validacion) * 0.3)  # 10% de la serie de validación
#
## Obtener índices aleatorios donde se eliminarán valores
#indices_faltantes = np.random.choice(serie_validacion.index, n_faltantes, replace=False)
#
## Crear una copia de la serie de validación y eliminar los valores en los índices seleccionados
#serie_con_nulos = serie_validacion.copy()
#serie_con_nulos.loc[indices_faltantes] = np.nan
#
## Definir el tamaño del bloque de valores faltantes
##block_size = int(len(serie_validacion) * 0.2)  # Un 10% de los datos en un bloque
##
### Seleccionar un índice inicial aleatorio para empezar el bloque
##inicio_bloque = np.random.randint(0, len(serie_validacion) - block_size)
##
### Crear la copia de la serie y eliminar un bloque de valores
##serie_con_nulos_bloque = serie_validacion.copy()
##serie_con_nulos_bloque.iloc[inicio_bloque:inicio_bloque + block_size] = np.nan
#
#def imputar_serie(serie_nulos):
#    
#    while serie_nulos.isna().sum() > 0:
#
#        next_nan_value_date = serie_nulos[serie_nulos.isna()].index[0]
#
#        model_df = serie_nulos.loc[serie_nulos.index <= next_nan_value_date]
#        model_df = pd.DataFrame(model_df)
#        lags = 3
#        for lag in range(1, lags + 1):
#            model_df[f'lag_{lag}'] = model_df[var].shift(lag)
#        next_nan_value = model_df.tail(1)
#        model_df = model_df.dropna()
#
#        X = model_df.drop(columns=[var])
#        y = model_df[var]
#
#        model = XGBRegressor()
#        model.fit(X, y)
#        y_pred = model.predict(next_nan_value.drop(columns=var))
#        next_nan_value[var] = y_pred.round(0)
#
#        serie_nulos.loc[next_nan_value_date] = next_nan_value[var].values[0]
#        #print(next_nan_value_date)
#    return serie_nulos
#
#serie_imputada = imputar_serie(serie_con_nulos.copy())
#
## Valores verdaderos (antes de eliminar)
#valores_reales = serie_validacion.loc[serie_con_nulos.isna() == True]
#
## Valores imputados
#valores_imputados = serie_imputada.loc[serie_con_nulos.isna() == True]
#
## Calcular el MAE, MSE y MAPE
#from sklearn.metrics import mean_absolute_error, mean_squared_error
#
## MAE
#mae = mean_absolute_error(valores_reales, valores_imputados)
#print(f"MAE: {mae:.2f}")
#
## MSE
#mse = mean_squared_error(valores_reales, valores_imputados)
#print(f"MSE: {mse:.2f}")
#
## MAPE
#mape = np.mean(np.abs((valores_reales - valores_imputados) / valores_reales)) * 100
#print(f"MAPE: {mape:.2f}%")